# pickle


# json


# dbm

- 用于小型程序中,不需要关系型数据库时,可以方便的用持久字典来存储键值对.
- dbm 和 python 中的字典非常类似。
- dbm 的键和值都必须是 str 或者 bytes 类型.

dbm.open,会打开（创建）一个文件,第一个参数直接传入文件名,第二个参数表示模式。常见的模式如下:

- r:可读,默认就是这个模式。
- w:可读可写。但是 r、w,都必须确保文件已经存在,否则报错。
- c:可读可写,文件不存在时会创建(平常的模式一般都会选择 c)
- n:可读可写,但总是会创建一个新的文件,也就是说如果创建同名文件,那么之前的内容都会被清空,也就是起不到追加的效果。


In [ ]:
import dbm

In [ ]:
db = dbm.open("store", "c")
db["name"] = "satori"
db["age"] = "16"
db["gender"] = "f"
db["anime"] = "11111111111"

db.close()

In [ ]:
db = dbm.open("store", "c")
print(db.keys())

db["anime"]

In [ ]:
for key in db.keys():
    print(f"key={key}, value={db[key]}")

# shelve

- 可以持久化任意对象,功能远比 dbm 强大

有一点值得注意：

- shelve 不记录对象的修改(例如 向保存在 shelve 中的 list 调用 append(),append 的内容不会被记录)
- 如果想把变更记录到 shelve 中,需要用重新赋值(=),或者给 shelve 指定 writeback 参数(会有额外的内存消耗,不推荐)


In [ ]:
import shelve

## 保存标准库对象


In [ ]:
# 参数flag默认是c,因此我们只需要传入文件名就可以了，这个是自动追加在后面的
sh = shelve.open("shelve1")

sh["dict"] = {"name": "satori", "age": 16}
sh["list"] = [1, 2, 3, 4]
sh["set"] = {1, 2, 3, 2}

sh.close()  # 写完之后关闭文件(关闭之后就无法操作了)，刷到内存里面

In [ ]:
sh2 = shelve.open("shelve1")
print(sh2["dict"], sh2["dict"].keys())

In [ ]:
print(sh2["list"], sum(sh2["list"]))

In [ ]:
print(sh2["set"])

In [ ]:
print(type(sh2["set"]))

In [ ]:
sh.close()

## 保存自定义对象，反序列化后调用方法


In [ ]:
sh2 = shelve.open("shelve2")


class A:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @property
    def print_info(self):
        return f"my name is {self.name}, age is {self.age}"


a = A("satori", 16)
sh2["A"] = A
sh2["a"] = a
sh2.close()

In [ ]:
sh2 = shelve.open("shelve2")
print(sh2["A"]("mashiro", "17").print_info)
print(sh2["a"].print_info)

## writeback---存储修改


In [ ]:
sh = shelve.open("shelve3")
sh["list"] = [1, 2, 3]
sh.close()

In [ ]:
# append无效,不会被保存
sh = shelve.open("shelve3")
sh["list"].append("xxxx")
sh.close()

sh = shelve.open("shelve3")
print(sh["list"])

In [ ]:
sh = shelve.open("shelve3", writeback=True)
sh["list"].append("xxxx")
sh.close()

sh = shelve.open("shelve3")
print(sh["list"])

## PersistentDict

In [ ]:
import shelve

class PersistentDict:
    def __init__(self, filename):
        self.filename = filename

    def __enter__(self):
        self.db = shelve.open(self.filename)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.db.close()

    def __getitem__(self, key):
        return self.db[key]

    def __setitem__(self, key, value):
        self.db[key] = value

    def __delitem__(self, key):
        del self.db[key]

    def __contains__(self, key):
        return key in self.db

    def keys(self):
        return [key for key in self.db.keys()]

    def values(self):
        return [value for value in self.db.values()]

    def items(self):
        return {key: self.db[str(key)] for key in self.db.keys()}

In [ ]:
with PersistentDict("mydict") as mydict:
    mydict["key1"] = "value1"
    mydict["key2"] = "value2"

In [ ]:
with PersistentDict("mydict") as mydict:
    print(mydict["key1"]) 
    print(mydict["key2"]) 

In [ ]:
with PersistentDict("mydict") as mydict:
    print(mydict.items())  

In [ ]:
with PersistentDict("mydict") as mydict:
    print(mydict.keys())

In [ ]:
with PersistentDict("mydict") as mydict:
    print(mydict.values())

## PersistentDict-Generic

In [1]:
from typing import TypeVar, Generic, Dict
import shelve

# 定义键和值的类型变量
K = TypeVar("K")
V = TypeVar("V")


# 定义持久化字典类，并添加范型
class PersistentDict(Generic[K, V]):
    def __init__(self, filename: str):
        self.filename = filename

    def __enter__(self) -> "PersistentDict":
        self.db = shelve.open(self.filename)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.db.close()

    def __getitem__(self, key: K) -> V:
        return self.db[str(key)]

    def __setitem__(self, key: K, value: V):
        self.db[str(key)] = value

    def __delitem__(self, key: K):
        del self.db[str(key)]

    def __contains__(self, key: K) -> bool:
        return str(key) in self.db

    def keys(self) -> list[K]:
        return [key for key in self.db.keys()]  # type: ignore

    def values(self) -> list[V]:
        return [value for value in self.db.values()]  # type: ignore

    def items(self) -> dict[K, V]:
        return {key: self.db[str(key)] for key in self.db.keys()}  # type: ignore

In [2]:
with PersistentDict[int, str]("persistent_dict") as pd:
    pd[1] = "value1"
    pd[2] = "value2"
    pd[3] = "value3"


In [3]:
with PersistentDict[int, str]("persistent_dict") as pd:
    print(pd.items())   

{'2': 'value2', '1': 'value1', '3': 'value3'}


In [4]:
with PersistentDict[int, str]("persistent_dict") as mydict:
    print(mydict.items())  

{'2': 'value2', '1': 'value1', '3': 'value3'}


In [5]:
with PersistentDict[int, str]("persistent_dict") as mydict:
    print(mydict.keys())

['2', '1', '3']


In [6]:
with PersistentDict[int, str]("persistent_dict") as mydict:
    print(mydict.values())

['value2', 'value1', 'value3']
